<a href="https://colab.research.google.com/github/Jarrydtb/proj5/blob/master/Project5_with_PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pandas as pd
import operator
from sklearn import preprocessing


In [132]:
##IMPORT FILE FOR DATA EXTRACTION
benign = pd.read_csv("/content/drive/Shared drives/Machine Learning Project 5/benign.csv")
malicious = pd.read_csv("/content/drive/Shared drives/Machine Learning Project 5/malicious.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#CREATE SINGLE DATAFRAME FOR BOTH THE BENIGN AND MALICIOUS DATA WITH MIXED ROWS
benign = benign.sample(frac=0.25,random_state=1)
malicious = malicious.sample(frac=0.25,random_state=1)
data = [benign,malicious]
mixedData = pd.concat(data)

In [0]:
##INSERT SECTION FOR DATA PRE-PROCESSING 
x = mixedData
labels = list(x.columns.values)

In [0]:
X=x.fillna(0)
X=X.replace(np.inf, 0)

In [136]:
print(len(X))

393727


In [137]:
print(X)

                                          Flow ID         Src IP  ... Idle Min Label
1045953                                         0              0  ...        0   ben
358409                                          0              0  ...        0   ben
875012                                          0              0  ...        0   ben
555046                                          0              0  ...        0   ben
272972           1.1.67.181-1.2.120.28-31957-80-6     1.2.120.28  ...        0   ben
...                                           ...            ...  ...      ...   ...
258342      147.32.84.165-69.30.193.210-1638-25-6  147.32.84.165  ...        0   mal
254000     147.32.84.193-173.13.139.154-4760-25-6  147.32.84.193  ...        0   mal
97805      147.32.84.165-216.157.130.15-8251-25-6  147.32.84.165  ...        0   mal
393111         10.0.2.109-23.63.92.101-52811-80-6     10.0.2.109  ...        0   mal
205829   147.32.84.165-67.171.127.111-4248-6667-6  147.32.84.165 

In [0]:
#IDENTIFY INDEXES OF LABELLED ENTRIES AND REMOVE THEM
temp = X.to_numpy()
index = []
count  = 0 
newArray = []
for i in temp:
  if "Src Port" in i:
    index.append(count)
  count += 1

val = 0 
for i in temp:
  if val not in index:
    newArray.append(i)
  val += 1

In [139]:
print(len(newArray))

393715


In [0]:
#CHECK TO MAKE SURE LABELLED ENTRIES ARE REMOVED
count = 0 
for i in newArray:
  if "Src Port" in i:
    print(count)
  count += 1

In [141]:
print(len(newArray))

393715


In [0]:
array = []
for i in newArray:
  if 'Infinity' in i:
    temp = []
    for j in i:
      if j == 'Infinity':
        temp.append(0)
      else:
        temp.append(j)
    array.append(temp)
  else:
    array.append(i)
newArray = array

In [143]:
print(len(newArray))

393715


In [0]:
#SLICE 'N DICE
#Remove all zeroed entries or entries with no Flow ID 
new = []
for i in newArray:
  if i[0] != 0:
    new.append(i)
newArray = new

In [145]:
print(len(newArray))

212594


In [0]:
df = pd.DataFrame(newArray,columns=labels)#Create new DF for data before removal and normalisation 
## RENAME DF TO X 

In [147]:
print(labels)

['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg', 'F

In [148]:
##DROP ATTRIBUTES FOR FEATURE SELECTION HERE 
df1 = df.drop(columns=labels[6]) # Timestamp
labels = list(df1.columns.values)
print(list(df1.columns.values))
df1 = df1.drop(columns=[labels[1],labels[3]]) # IP Addresses
labels = list(df1.columns.values)
print("Remaining Lables:\n",labels)

['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg', 'Fwd Pkts/b Avg

In [0]:
#ids = X[labels[0]]
Y = df1[labels[-1]]
ids = df1[labels[0]]
X1 = df1.drop(columns=labels[0]) # Flow ID --> USE FOR PUTTING DATA TOGETHER FOR TIME SERIES
X1 = X1.drop(columns=labels[-1])

In [0]:
newY = []
for i in Y:
  if i == 'ben':
    newY.append(0)
  else:
    newY.append(1)

In [0]:
labelsNew = list(X1.columns.values)
for col in labelsNew:
  X1[col] = pd.to_numeric(X1[col])

In [219]:
print(X1.iloc[49559])

Src Port         63205.0
Dst Port            21.0
Protocol             6.0
Flow Duration       15.0
Tot Fwd Pkts         0.0
                  ...   
Active Min           0.0
Idle Mean            0.0
Idle Std             0.0
Idle Max             0.0
Idle Min             0.0
Name: 49559, Length: 79, dtype: float64


In [0]:
#NORMALISE DATA
# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(X1)

# Run the normalizer on the dataframe
df_normalized = pd.DataFrame(x_scaled,columns=labelsNew)

In [0]:
newYdf = pd.DataFrame(newY,columns=['Label'])

In [227]:
X = pd.concat([ids,df_normalized,newYdf],axis=1,sort=False) 
print(X)

                                         Flow ID  Src Port  ...  Idle Min  Label
0               1.1.67.181-1.2.120.28-31957-80-6  0.001221  ...       0.0      0
1             1.1.234.194-1.2.209.247-54990-21-6  0.839094  ...       0.0      0
2                1.1.15.97-1.2.162.128-6998-21-6  0.000320  ...       0.0      0
3              1.1.136.17-1.2.247.160-18535-21-6  0.282826  ...       0.0      0
4             1.1.143.211-1.2.67.36-18402-3306-6  0.050446  ...       0.0      0
...                                          ...       ...  ...       ...    ...
212589     147.32.84.165-69.30.193.210-1638-25-6  0.024994  ...       0.0      1
212590    147.32.84.193-173.13.139.154-4760-25-6  0.072633  ...       0.0      1
212591    147.32.84.165-216.157.130.15-8251-25-6  0.125902  ...       0.0      1
212592        10.0.2.109-23.63.92.101-52811-80-6  0.805844  ...       0.0      1
212593  147.32.84.165-67.171.127.111-4248-6667-6  0.064820  ...       0.0      1

[212594 rows x 81 columns]


In [0]:
cont = {}
for i in X[labels[0]]:
  if cont.get(i):
    val = cont.get(i)
    val = val + 1
    cont.update(i = val)
  else:
    cont[i] = 1 

In [229]:
highestVal = 0 
for keys, values in cont.items():
  if values > highestVal:
    highestVal = values
  else:
    continue 
print(highestVal)
paddingVal = highestVal
numCol = len(X.columns.values) - 1 

2


In [230]:
print(paddingVal)
print(numCol)
print(len(X))

2
80
212594


In [0]:
idsUnique = []
for keys in cont.keys():
  idsUnique.append(keys)

In [232]:
%%time 
#CREATE INPUT DATA 'IMAGES' #size = 2 x N: N number of columns 
df_temp = X.groupby(labels[0])
grouped = list(list(df_temp)) #List of Dataframes in their respective groups

CPU times: user 47.7 s, sys: 549 ms, total: 48.2 s
Wall time: 48.2 s


In [0]:
images = []
for i in grouped:
  df_temp = i[1]
  images.append(df_temp.iloc[:,1:].values)

In [237]:
%%time
padding = np.full((numCol-1), 0).astype('float')
newImages = []
for i in images:
  flag = i[0][-1]
  if flag == 'mal':
    flag = 1
  else:
    flag = 0
  NewPad = np.append(padding,[flag])
  if len(i) < highestVal:
    for j in range(highestVal-len(i)):
      if j == 0:  
        x = np.append(i,[NewPad],axis=0)
      else:
        x = np.append(x,[NewPad],axis=0)
    newImages.append(x)
  else:
    newImages.append(i)
  count += 1

CPU times: user 3.24 s, sys: 42.7 ms, total: 3.28 s
Wall time: 3.28 s


In [0]:
##Seperate Y and X values
mainX = []
mainY = []
for matrix in newImages:
  tempMatrix = []
  tempY = []
  for j in matrix:
    tempMatrix.append(j[:-2])
    tempY.append(j[-1])
  mainX.append(tempMatrix)
  mainY.append(tempY[0])

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2) ##ADJUST 'test_size' ACCORDINGLY 

In [0]:
input_shape = x_train.shape ##TODO: GET INPUT SHAPE FROM PRE-PROCESSING STAGE
print(x_train.shape)

In [0]:
def cnn(x_train, y_train, x_test, y_test):
    num_classes = 2
    batch_size = 256
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                    activation='relu',
                    input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizers.Adam(),
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print(model.summary())
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [0]:
def rnn(x_train, y_train, x_test, y_test):
  return 0

In [0]:
cnn(x_train, y_train, x_test, y_test)